# Run Full TSB-CL on Kaggle (Using GitHub Code)

This notebook is designed to run on Kaggle. It performs the following steps:
1.  **Clones the GitHub repository** containing the project code.
2.  **Compiles the MSBE** (Maximal Similar Biclique Enumeration) C++ tool.
3.  **Runs the training script** by importing the code directly from the cloned repository.

This ensures you are always running the latest version of the code without manually copying large blocks of text.

In [ ]:
# 1. Clone Repository
# Remove existing folder to ensure fresh clone
!rm -rf ai_project 
!git clone https://github.com/yangzeha/ai_project.git

In [ ]:
# 2. Compile MSBE (C++ Tool)
import os
import subprocess

# Define paths
PROJECT_DIR = os.path.abspath("ai_project")
MSBE_DIR = os.path.join(PROJECT_DIR, "Similar-Biclique-Idx-main")
MSBE_EXE_PATH = os.path.join(MSBE_DIR, "msbe")

# Check if directory exists
if os.path.exists(MSBE_DIR):
    print(f"Compiling MSBE in {MSBE_DIR}...")
    
    # Clean up any existing 'msbe' file/folder to avoid conflicts
    if os.path.exists(MSBE_EXE_PATH):
        if os.path.isdir(MSBE_EXE_PATH):
            print("Removing existing 'msbe' directory...")
            subprocess.run(["rm", "-rf", "msbe"], cwd=MSBE_DIR)
        else:
            print("Removing existing 'msbe' file...")
            os.remove(MSBE_EXE_PATH)

    # Compile command
    cmd = ["g++", "-O3", "main.cpp", "-o", "msbe", "-D_PrintResults_"]
    try:
        subprocess.run(cmd, cwd=MSBE_DIR, check=True)
        print("✅ Compilation successful!")
        
        # Grant execute permissions (Crucial for Linux/Kaggle)
        subprocess.run(["chmod", "+x", "msbe"], cwd=MSBE_DIR, check=True)
        print("✅ Execute permissions granted.")
        
        # Verify executable exists
        if os.path.exists(MSBE_EXE_PATH):
             print(f"Executable created at: {MSBE_EXE_PATH}")
        else:
             print("Warning: 'msbe' file not found after compilation.")
             
    except subprocess.CalledProcessError as e:
        print(f"❌ Compilation failed: {e}")
else:
    print(f"❌ Error: Directory {MSBE_DIR} not found. Did git clone succeed?")

In [ ]:
# 3. Run Training
import sys
import os

# Add project root to system path so we can import modules
PROJECT_DIR = os.path.abspath("ai_project")
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

# Import modules from the cloned repository
try:
    from TSB_CL_Project.solo_model.model_variants import FullTSBCL
    from TSB_CL_Project.solo_model import train_utils
    print("✅ Modules imported successfully.")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Current sys.path:", sys.path)
    print("Directory structure:", os.listdir(PROJECT_DIR))

# Configure Training Parameters
# We override the default settings in train_utils
train_utils.NUM_SNAPSHOTS = 5  # Use 5 snapshots (80% train, 20% test)
train_utils.EPOCHS = 10        # Train for 10 epochs
train_utils.BATCH_SIZE = 2048

# Override MSBE_EXE path to ensure it's absolute and correct
MSBE_DIR = os.path.join(PROJECT_DIR, "Similar-Biclique-Idx-main")
train_utils.MSBE_EXE = os.path.join(MSBE_DIR, "msbe")
print(f"Using MSBE executable at: {train_utils.MSBE_EXE}")

# Run the training loop
print("\n🚀 Starting Training for Full TSB-CL...")
metrics = train_utils.run_training(
    model_class=FullTSBCL,
    model_name="Full TSB-CL (GitHub Version)",
    model_type="full",
    epochs=train_utils.EPOCHS,
    tau=2,
    epsilon=0.1
)

print("\nTraining Complete!")
print("Final Recall:", metrics['recall'][-1])
print("Final NDCG:", metrics['ndcg'][-1])